In [1]:
import pandas as pd
import numpy as np
from scipy.interpolate import griddata
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut

In [2]:
# Set random seed for reproducibility
np.random.seed(42)# Define the boundaries of Texas
lat_min, lat_max = 25.8371, 36.5007
lon_min, lon_max = -106.6456, -93.5083# Generate a grid of points across Texas
n_points = 1000


latitudes = np.random.uniform(lat_min, lat_max, n_points)
longitudes = np.random.uniform(lon_min, lon_max, n_points)# Create a dataframe
df = pd.DataFrame({
    'latitude': latitudes,
    'longitude': longitudes
})

In [3]:
# Function to get elevation data (this would ideally use a DEM dataset)
def get_elevation(lat, lon):
    # This is a simplified model. In reality, you'd use a Digital Elevation Model.
    return np.random.uniform(0, 1000)  # Texas elevations generally range from 0 to about 2,667 meters
df['elevation'] = df.apply(lambda row: get_elevation(row['latitude'], row['longitude']), axis=1)

# Magnetic field strength (nT)
# Data based on NOAA's magnetic field calculator
mag_field_min, mag_field_max = 47000, 52000
df['magnetic_field'] = np.random.uniform(mag_field_min, mag_field_max, n_points)# Gravity anomalies (mGal)

# Data based on the World Gravity Map
gravity_min, gravity_max = -40, 40
df['gravity_anomaly'] = np.random.uniform(gravity_min, gravity_max, n_points)# Atmospheric pressure (hPa)

# Pressure varies with elevation
def calculate_pressure(elevation):
    p0 = 1013.25  # sea level standard atmosphere in hPa
    return p0 * (1 - 0.0065 * elevation / 288.15) ** 5.255
df['pressure'] = df['elevation'].apply(calculate_pressure)

In [4]:
df.head()

,latitude,longitude,elevation,magnetic_field,gravity_anomaly,pressure
0,29.831046,-104.213453,261.705684,50363.514971,5.759670,982.208453
1,35.975137,-99.526485,246.978799,50983.406986,24.434586,983.934538
2,33.642791,-95.177449,906.254581,48252.339494,20.812874,909.028954
3,32.220955,-97.026142,249.546200,50124.370498,-27.688008,983.633446
4,27.500820,-96.049564,271.949726,49858.729916,-28.060042,981.009232


In [5]:
# Add some spatial correlation
def add_spatial_correlation(series, strength=0.5):
    grid_x, grid_y = np.mgrid[lat_min:lat_max:100j, lon_min:lon_max:100j]
    points = df[['latitude', 'longitude']].values
    values = series.values
    grid_z = griddata(points, values, (grid_x, grid_y), method='cubic')
    interpolated = griddata((grid_x.ravel(), grid_y.ravel()), grid_z.ravel(), points, method='nearest')
    return series * (1 - strength) + interpolated * strength

df['magnetic_field'] = add_spatial_correlation(df['magnetic_field'])
df['gravity_anomaly'] = add_spatial_correlation(df['gravity_anomaly'])# Attempt to get real place names (this may take some time and might hit API limits)

In [6]:
df.head()

,latitude,longitude,elevation,magnetic_field,gravity_anomaly,pressure
0,29.831046,-104.213453,261.705684,50578.130565,7.044395,982.208453
1,35.975137,-99.526485,246.978799,50938.755317,25.550456,983.934538
2,33.642791,-95.177449,906.254581,48525.848269,18.973367,909.028954
3,32.220955,-97.026142,249.546200,49937.069959,-28.628942,983.633446
4,27.500820,-96.049564,271.949726,50021.341612,-30.246191,981.009232


In [7]:
# Attempt to get real place names (this may take some time and might hit API limits)
#geolocator = Nominatim(user_agent="alis_texas_data")

#def get_place_name(lat, lon, retries=3):
  #  global geolocator  # Ensure geolocator is accessed globally
    
   # try:
    #    location = geolocator.reverse(f"{lat}, {lon}", timeout=10)  # Adjust timeout if necessary
     #   if location is not None and 'address' in location.raw:
     #       return location.raw['address'].get('city', 'Unknown')
     #   else:
     #       return 'Unknown'
   # except GeocoderTimedOut as e:
    #    print(f"Geocoder service timed out: {e}")
    #    if retries > 0:
     #       print(f"Retrying after 2 seconds... Attempts left: {retries}")
      #      time.sleep(2)
      #      return get_place_name(lat, lon, retries=retries - 1)
       # else:
        #    print("Maximum retries reached. Returning 'Unknown'.")
         #   return 'Unknown'
   # except GeocoderServiceError as e:
    #    print(f"Geocoder service error: {e}")
     #   return 'Unknown'
    #except Exception as e:
     #   print(f"Error retrieving place name: {str(e)}")
      #  return 'Unknown'
    
#df['place_name'] = df.apply(lambda row: get_place_name(row['latitude'], row['longitude']), axis=1)

In [8]:
# Save to CSV
df_filled = df.fillna(df.mean())
df_filled.to_csv('texas_alis_data.csv', index=False)
#print(df.head())
print(df_filled.describe())

          latitude    longitude    elevation  magnetic_field  gravity_anomaly  \
count  1000.000000  1000.000000  1000.000000     1000.000000      1000.000000   
mean     31.065000   -99.984762   502.405726    49457.477088        -0.485975   
std       3.115236     3.838586   290.674195     1387.993538        22.802862   
min      25.886494  -106.603321     0.011635    45904.245531       -64.595799   
25%      28.353425  -103.478535   261.350977    48298.568766       -19.293227   
50%      31.134855   -99.830837   500.613920    49457.477088        -0.485975   
75%      33.774226   -96.655142   759.103527    50590.359808        17.973152   
max      36.497689   -93.516002   997.820856    52488.391389        55.213678   

          pressure  
count  1000.000000  
mean    954.811500  
std      33.279723  
min     899.001220  
25%     925.333551  
50%     954.547618  
75%     982.250000  
max    1013.248603  


In [9]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors


class AnomalyPredictor:
    def __init__(self):
        self.model = None
        self.scaler_X = StandardScaler()
        self.scaler_y = StandardScaler()

    def create_model(self):
        model = keras.Sequential([
            keras.layers.Dense(64, activation='relu', input_shape=(3,)),
            keras.layers.Dense(32, activation='relu'),
            keras.layers.Dense(16, activation='relu'),
            keras.layers.Dense(2)  # Output layer: [magnetic_anomaly, gravity_anomaly]
        ])
        
        model.compile(optimizer='adam', loss='mse', metrics=['mae'])
        self.model = model

    def train(self, X, y, epochs=50, batch_size=64):
        X_scaled = self.scaler_X.fit_transform(X)
        y_scaled = self.scaler_y.fit_transform(y)
        
        X_train, X_val, y_train, y_val = train_test_split(X_scaled, y_scaled, test_size=0.3, random_state=42)
        
        history = self.model.fit(
            X_train, y_train,
            validation_data=(X_val, y_val),
            epochs=epochs,
            batch_size=batch_size,
            verbose=1
        )
        return history

    def predict(self, X):
        X_scaled = self.scaler_X.transform(X)
        y_pred_scaled = self.model.predict(X_scaled)
        return self.scaler_y.inverse_transform(y_pred_scaled)


In [10]:
class MultiDimensionalVAP:
    def __init__(self, dataframe):
        self.data = dataframe
        self.nbrs = NearestNeighbors(n_neighbors=3, algorithm='ball_tree').fit(self.data[['latitude', 'longitude']])

    def add_vap(self, latitude, longitude, elevation, magnetic_field, gravity_anomaly, pressure):
        new_vap = pd.DataFrame({
            'latitude': [latitude],
            'longitude': [longitude],
            'elevation': [elevation],
            'magnetic_field': [magnetic_field],
            'gravity_anomaly': [gravity_anomaly],
            'pressure': [pressure]
        })
        self.data = pd.concat([self.data, new_vap], ignore_index=True)
        self.nbrs.fit(self.data[['latitude', 'longitude']])

    def find_nearest_vaps(self, latitude, longitude, num_vaps=3):
        distances, indices = self.nbrs.kneighbors([[latitude, longitude]], n_neighbors=num_vaps)
        return self.data.iloc[indices[0]]
    
    def interpolate_position(self, vaps, magnetic_reading, gravity_reading, pressure_reading):
        """
        Interpolates the position based on given readings.
        
        Parameters:
        vaps (pd.DataFrame): DataFrame containing VAPs data.
        magnetic_reading (float): Magnetic field reading.
        gravity_reading (float): Gravity anomaly reading.
        pressure_reading (float): Atmospheric pressure reading.
        
        Returns:
        dict: A dictionary containing the interpolated position and readings.
        """
        # Placeholder implementation
        # You can add more sophisticated logic here based on your application requirements
        
        # For this example, just returning a simple dictionary with the provided readings
        
        return {
            'latitude': vaps.iloc[0]['latitude'],
            'longitude': vaps.iloc[0]['longitude'],
            'elevation': vaps.iloc[0]['elevation'],
            'magnetic_field': magnetic_reading,
            'gravity_anomaly': gravity_reading,
            'pressure': pressure_reading
        }

In [11]:
class EnhancedMultiDimensionalVAP(MultiDimensionalVAP):
    def __init__(self, db_path):
        super().__init__(db_path)
        self.anomaly_predictor = AnomalyPredictor()
        self.train_predictor()

    def train_predictor(self):
        # Fetch all VAP data
        #self.cursor.execute('SELECT latitude, longitude, magnetic_anomaly, gravity_anomaly FROM multi_vaps')
        data = self.data[['latitude', 'longitude', 'elevation', 'magnetic_field', 'gravity_anomaly']]
        X = data[['latitude', 'longitude', 'elevation']].values
        y = data[['magnetic_field', 'gravity_anomaly']].values
        
        self.anomaly_predictor.create_model()
        self.anomaly_predictor.train(X, y)

    def predict_anomalies(self, lat, lon, elevation):
        X = np.array([[lat, lon, elevation]])
        y_pred = self.anomaly_predictor.predict(X)
        return y_pred[0]
        
    def interpolate_position(self, vaps, magnetic_reading, gravity_reading, pressure_reading):
        if len(vaps) < 3:  # If we have very sparse data
            # Predict anomalies for the current location
            lat, lon = vaps.iloc[0]['latitude'], vaps.iloc[0]['longitude']  # Use the nearest VAP's location as an approximation
            elevation = 100  # Assume a constant elevation for simplicity
            predicted_anomalies = self.predict_anomalies(lat, lon, elevation)

            # Create a DataFrame with the new row
            new_row = pd.DataFrame({
                'latitude': [lat],
                'longitude': [lon],
                'elevation': [elevation],
                'magnetic_field': [predicted_anomalies[0]],
                'gravity_anomaly': [predicted_anomalies[1]],
                'pressure': [pressure_reading],
                'some_other_field': [0.5]  # Adjust as necessary
            })

            # Combine the original DataFrame with the new row
            combined_vaps = pd.concat([vaps, new_row], ignore_index=True)

            return super().interpolate_position(combined_vaps, magnetic_reading, gravity_reading, pressure_reading)
        else:
            return super().interpolate_position(vaps, magnetic_reading, gravity_reading, pressure_reading)
        
    def calculate_confidence(self, vaps, magnetic_reading, gravity_reading, pressure_reading):
        """
        Calculate the confidence of the interpolation based on the VAPs data and the readings.
        
        Parameters:
        vaps (pd.DataFrame): DataFrame containing VAPs data.
        magnetic_reading (float): Magnetic field reading.
        gravity_reading (float): Gravity anomaly reading.
        pressure_reading (float): Atmospheric pressure reading.
        
        Returns:
        float: Confidence value (0 to 1).
        """
        if len(vaps) < 2:
            return 0.5  # If less than 2 VAPs, return a low confidence by default
        
        # Calculate distance-based confidence
        lat_mean = vaps['latitude'].mean()
        lon_mean = vaps['longitude'].mean()
        distances = np.sqrt((vaps['latitude'] - lat_mean)**2 + (vaps['longitude'] - lon_mean)**2)
        distance_confidence = 1 - np.mean(distances)  # Closer VAPs give higher confidence
        
        # Calculate reading variance-based confidence
        magnetic_variance = vaps['magnetic_field'].var()
        gravity_variance = vaps['gravity_anomaly'].var()
        pressure_variance = vaps['pressure'].var()
        reading_variance = (magnetic_variance + gravity_variance + pressure_variance) / 3
        reading_confidence = 1 / (1 + reading_variance)  # Lower variance gives higher confidence
        
        # Combine both confidences
        combined_confidence = (distance_confidence + reading_confidence) / 2
        
        # Ensure confidence is between 0 and 1
        return max(0, min(1, combined_confidence))


In [12]:
# Usage example
vap_system = EnhancedMultiDimensionalVAP(df_filled)

Epoch 1/50
11/11 [==============================] - 0s 9ms/step - loss: 1.0034 - mae: 0.8558 - val_loss: 1.0189 - val_mae: 0.8572
Epoch 2/50
11/11 [==============================] - 0s 2ms/step - loss: 0.9922 - mae: 0.8503 - val_loss: 1.0167 - val_mae: 0.8565
Epoch 3/50
11/11 [==============================] - 0s 2ms/step - loss: 0.9889 - mae: 0.8489 - val_loss: 1.0162 - val_mae: 0.8562
Epoch 4/50
11/11 [==============================] - 0s 2ms/step - loss: 0.9863 - mae: 0.8476 - val_loss: 1.0161 - val_mae: 0.8558
Epoch 5/50
11/11 [==============================] - 0s 2ms/step - loss: 0.9852 - mae: 0.8473 - val_loss: 1.0181 - val_mae: 0.8567
Epoch 6/50
11/11 [==============================] - 0s 2ms/step - loss: 0.9835 - mae: 0.8467 - val_loss: 1.0174 - val_mae: 0.8566
Epoch 7/50
11/11 [==============================] - 0s 2ms/step - loss: 0.9812 - mae: 0.8457 - val_loss: 1.0175 - val_mae: 0.8570
Epoch 8/50
11/11 [==============================] - 0s 2ms/step - loss: 0.9806 - mae: 0.84

In [13]:
# Add some sample VAPs (this would typically be done with real data)
vap_system.add_vap(37.7749, -122.4194, 100, 50000, 8,1013)
vap_system.add_vap(37.7750, -122.4195, 110, 55000, 7,1012)
vap_system.add_vap(37.7748, -122.4193, 90, 45000, 9, 1014)

In [14]:
#df_filled.head(30)

In [15]:
# Find nearest VAPs
nearest_vaps = vap_system.find_nearest_vaps(37.7751, -122.4196, num_vaps=2)  # Intentionally using only 2 VAPs to simulate sparse data

/Users/juanmanuelpaiba/.pyenv/versions/3.8.17/lib/python3.8/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


In [16]:
nearest_vaps

,latitude,longitude,elevation,magnetic_field,gravity_anomaly,pressure
1001,37.7750,-122.4195,110.0,55000.0,7.0,1012.0
1000,37.7749,-122.4194,100.0,50000.0,8.0,1013.0


In [17]:
# Interpolate position
estimated_position = vap_system.interpolate_position(nearest_vaps, 52000, 8, 1013)

1/1 [==============================] - 0s 39ms/step


In [18]:
estimated_position

{'latitude': 37.775,
 'longitude': -122.4195,
 'elevation': 110.0,
 'magnetic_field': 52000,
 'gravity_anomaly': 8,
 'pressure': 1013}

In [19]:
# Calculate confidence
confidence = vap_system.calculate_confidence(nearest_vaps, 52000, 8, 1013)

In [20]:
print(f"Estimated position: {estimated_position}")
print(f"Confidence: {confidence}")

# Predict anomalies for a new location
predicted_anomalies = vap_system.predict_anomalies(37.7752, -122.4197, 100)
print(f"Predicted anomalies: Magnetic = {predicted_anomalies[0]}, Gravity = {predicted_anomalies[1]}")


Estimated position: {'latitude': 37.775, 'longitude': -122.4195, 'elevation': 110.0, 'magnetic_field': 52000, 'gravity_anomaly': 8, 'pressure': 1013}
Confidence: 0.49996476466090234
1/1 [==============================] - 0s 10ms/step
Predicted anomalies: Magnetic = 49866.234375, Gravity = -10.199071884155273
